In [ ]:
# Import Required Libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import json
import pickle
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
sys.path.insert(0, os.getcwd())
from data_loading import WaferDataLoader
from utility import (setup_model_and_loaders, hyperparameter_tuning, 
                     evaluate_model, train_model)
from models import MLP
from config import MLP_TUNING_GRID

# Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✓ All libraries imported successfully!")
print(f"Device: {device}")

## 1. Load and Prepare Data

In [ ]:
# Load data using WaferDataLoader
print("Loading wafer defect dataset...")
loader = WaferDataLoader()

print(f"✓ Dataset loaded successfully!")
print(f"X shape: {loader.X.shape}")
print(f"y shape: {loader.y.shape}")
print(f"Number of classes: {loader.num_classes}")

# Get the data
X = loader.X.astype('float32')
y = loader.y

# Normalize data
X_min, X_max = X.min(), X.max()
X_normalized = (X - X_min) / (X_max - X_min) if X_max > X_min else X

print(f"\n✓ Normalization:")
print(f"  Original range: [{X_min}, {X_max}]")
print(f"  Normalized range: [{X_normalized.min():.4f}, {X_normalized.max():.4f}]")

# Split into train/val/test (70% / 15% / 15%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_normalized, y, test_size=0.3, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"\n✓ Data split complete:")
print(f"  Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(y)*100:.1f}%)")
print(f"  Validation set: {X_val.shape[0]} samples ({X_val.shape[0]/len(y)*100:.1f}%)")
print(f"  Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(y)*100:.1f}%)")

## 2. Hyperparameter Tuning Grid

In [ ]:
# Display tuning grid
print("="*80)
print("MLP - HYPERPARAMETER TUNING GRID".center(80))
print("="*80)
print(f"\nTuning Grid:")
for param, values in MLP_TUNING_GRID.items():
    print(f"  {param}: {values}")

total_combinations = np.prod([len(v) for v in MLP_TUNING_GRID.values()])
print(f"\nTotal combinations to evaluate: {total_combinations}")
print("="*80)

## 3. Run Hyperparameter Tuning

In [ ]:
# MLP Hyperparameter Tuning
print("\n" + "="*80)
print("STARTING HYPERPARAMETER TUNING".center(80))
print("="*80)

mlp_results = hyperparameter_tuning(
    MLP, X_train, X_val, X_test, y_train, y_val, y_test,
    param_grid=MLP_TUNING_GRID,
    input_size=2704,
    num_classes=38,
    device=str(device),
    num_epochs=25,
    patience=5,
    verbose=True
)

print("\n" + "="*80)
print("TUNING COMPLETE".center(80))
print("="*80)

# Extract top 5
mlp_top5 = mlp_results['summary_df'].head(5).copy()
print("\nTop 5 Configurations:")
print(mlp_top5[['learning_rate', 'batch_size', 'hidden_sizes', 'dropout', 'num_epochs', 'optimizer', 'Val_Acc', 'Test_Acc']].to_string())

## 4. Validation Loss Curves - Top 5

In [ ]:
# Retrain top 5 models to get training histories
print("\nRetraining top 5 models to generate loss curves...")

def retrain_with_history(model_class, X_train, X_val, X_test, y_train, y_val, y_test, 
                         params, input_size, num_classes, device_str):
    """Retrain a model with given params and return training history"""
    try:
        # Extract parameters
        learning_rate = params.get('learning_rate', 0.001)
        batch_size = params.get('batch_size', 64)
        optimizer_type = params.get('optimizer', 'adam')
        epochs_to_train = params.get('num_epochs', 20)
        
        # Build model_kwargs for architecture parameters
        model_kwargs = {}
        if 'hidden_sizes' in params:
            hidden_sizes = params['hidden_sizes']
            if isinstance(hidden_sizes, str):
                hidden_sizes = hidden_sizes.strip('[]').split(',')
                hidden_sizes = [int(h.strip()) for h in hidden_sizes]
            model_kwargs['hidden_sizes'] = hidden_sizes
        
        if 'dropout' in params:
            model_kwargs['dropout'] = params['dropout']
        
        # Setup model and loaders
        setup_result = setup_model_and_loaders(
            model_class, X_train, X_val, X_test, y_train, y_val, y_test,
            input_size=input_size, num_classes=num_classes, device=device_str,
            batch_size=batch_size, model_kwargs=model_kwargs, verbose=False
        )
        
        model = setup_result['model']
        train_loader = setup_result['train_loader']
        val_loader = setup_result['val_loader']
        
        # Setup optimizer
        if optimizer_type.lower() == 'adam':
            opt = optim.Adam(model.parameters(), lr=learning_rate)
        else:
            opt = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
        
        # Train
        criterion = nn.CrossEntropyLoss()
        history = train_model(
            model, train_loader, val_loader, criterion, opt,
            num_epochs=epochs_to_train, device=device_str, patience=5
        )
        
        return history
    except Exception as e:
        print(f"Error: {e}")
        return None

# Get histories for top 5
top5_histories = []
for i, (_, row) in enumerate(mlp_top5.iterrows(), 1):
    print(f"  Retraining rank {i}/5...")
    
    # Extract parameters from row
    params = {
        'learning_rate': row['learning_rate'],
        'batch_size': int(row['batch_size']),
        'optimizer': row['optimizer'],
        'num_epochs': int(row['num_epochs']),
        'hidden_sizes': row['hidden_sizes'],
        'dropout': row['dropout'],
    }
    
    history = retrain_with_history(
        MLP, X_train, X_val, X_test, y_train, y_val, y_test,
        params, input_size=2704, num_classes=38, device_str=str(device)
    )
    if history:
        top5_histories.append(history)

print(f"✓ Successfully generated {len(top5_histories)} training histories")

In [ ]:
# Plot validation loss curves for top 5
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('Multi-Layer Perceptron - Top 5 Configurations\nTraining vs Validation Loss', 
             fontsize=14, fontweight='bold')

for idx, history in enumerate(top5_histories):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col]
    
    epochs = range(1, len(history['train_loss']) + 1)
    ax.plot(epochs, history['train_loss'], 'b-', label='Training Loss', linewidth=2, marker='o', markersize=4)
    ax.plot(epochs, history['val_loss'], 'r-', label='Validation Loss', linewidth=2, marker='s', markersize=4)
    
    rank = idx + 1
    val_acc = mlp_top5.iloc[idx]['Val_Acc']
    ax.set_title(f'Rank {rank} - Val Acc: {val_acc:.4f}', fontweight='bold', fontsize=11)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)

# Hide unused subplot
axes[1, 2].set_visible(False)

plt.tight_layout()
plt.savefig('mlp_validation_loss_curves.png', dpi=300, bbox_inches='tight')
print("✓ Loss curves plot saved as 'mlp_validation_loss_curves.png'")
plt.show()

## 5. Save Results

In [ ]:
# Create results directory
results_dir = 'mlp_results'
os.makedirs(results_dir, exist_ok=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

print("\n" + "="*80)
print("SAVING RESULTS".center(80))
print("="*80)

# 1. Save summary CSV with all results
mlp_results['summary_df'].to_csv(f'{results_dir}/all_results_{timestamp}.csv', index=False)
print(f"✓ All results saved to: all_results_{timestamp}.csv")

# 2. Save top 5 CSV
mlp_top5.to_csv(f'{results_dir}/top5_results_{timestamp}.csv', index=False)
print(f"✓ Top 5 results saved to: top5_results_{timestamp}.csv")

# 3. Save best model info
best_model_info = {
    'best_params': mlp_results['best_params'],
    'best_val_acc': float(mlp_results['best_val_acc']),
    'best_test_acc': float(mlp_results['best_test_acc']),
}

with open(f'{results_dir}/best_model_{timestamp}.json', 'w') as f:
    json.dump(best_model_info, f, indent=4, default=str)
print(f"✓ Best model info saved to: best_model_{timestamp}.json")

# 4. Save best model itself
torch.save(mlp_results['best_model'].state_dict(), 
           f'{results_dir}/best_model_weights_{timestamp}.pt')
print(f"✓ Best model weights saved to: best_model_weights_{timestamp}.pt")

# 5. Save training history for best model
with open(f'{results_dir}/best_model_history_{timestamp}.pkl', 'wb') as f:
    pickle.dump(mlp_results['best_history'], f)
print(f"✓ Best model training history saved to: best_model_history_{timestamp}.pkl")

# 6. Save training histories for top 5
for i, history in enumerate(top5_histories, 1):
    with open(f'{results_dir}/rank_{i:02d}_history_{timestamp}.pkl', 'wb') as f:
        pickle.dump(history, f)
print(f"✓ Top 5 training histories saved")

# 7. Save loss curves data as CSV for easy access
for i, history in enumerate(top5_histories, 1):
    loss_df = pd.DataFrame({
        'Epoch': range(1, len(history['train_loss']) + 1),
        'Train_Loss': history['train_loss'],
        'Val_Loss': history['val_loss'],
        'Train_Acc': history['train_acc'],
        'Val_Acc': history['val_acc'],
    })
    loss_df.to_csv(f'{results_dir}/rank_{i:02d}_loss_curves_{timestamp}.csv', index=False)
print(f"✓ Loss curves data saved as CSV for all top 5")

# 8. Save summary report
summary = {
    'model': 'MLP',
    'timestamp': timestamp,
    'total_combinations': int(total_combinations),
    'best_val_accuracy': float(mlp_results['best_val_acc']),
    'best_test_accuracy': float(mlp_results['best_test_acc']),
    'best_hyperparameters': mlp_results['best_params'],
    'top5_accuracies': mlp_top5['Val_Acc'].tolist(),
}

with open(f'{results_dir}/summary_report_{timestamp}.json', 'w') as f:
    json.dump(summary, f, indent=4, default=str)
print(f"✓ Summary report saved to: summary_report_{timestamp}.json")

print("\n" + "="*80)
print(f"All results saved to: {results_dir}/".center(80))
print("="*80)

## 6. Results Summary

In [ ]:
print("\n" + "="*80)
print("MULTI-LAYER PERCEPTRON - TUNING RESULTS SUMMARY".center(80))
print("="*80)

print(f"\nBest Model Performance:")
print(f"  Validation Accuracy: {mlp_results['best_val_acc']:.4f}")
print(f"  Test Accuracy: {mlp_results['best_test_acc']:.4f}")

print(f"\nBest Hyperparameters:")
for key, value in mlp_results['best_params'].items():
    print(f"  {key}: {value}")

print(f"\nTop 5 Validation Accuracies:")
for rank, acc in enumerate(mlp_top5['Val_Acc'].values, 1):
    print(f"  Rank {rank}: {acc:.4f}")

print(f"\nTotal tuning combinations evaluated: {total_combinations}")
print("\n" + "="*80)